In [1]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special

import random

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

from matplotlib import pyplot as plt

from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

In [2]:
# the analytical representation of exact solution
def heat_equ_analytical_solu(x, t):
    return np.sin(np.pi * x) * np.exp(-np.power(np.pi, 2) * t)

In [3]:
# ==========
# helping methods
# ==========

# generate a list from lower and upper bound
def gen_list(p0, pn, delta, dig=5):
    ret = []
    i = p0
    while i < pn:
        ret.append(float(i))
        i += delta
        i = round(i, dig)
    return ret

# padding and zero padding
def padding(origin, a_list, b_list):
    return np.hstack((a_list, origin, b_list))

def zero_padding(origin, num):
    zero_list = [0 for i in range(num)]
    return padding(origin, zero_list, zero_list)

In [4]:
# ==============
# getting trainning data
# ==============

# trainning pairs
def gen_pair(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-1-length)
        train = current_t[p-length:p+length+1]
        solu = next_t[p]
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in average)
def gen_pair_ave(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-2-length)
        train0 = current_t[p-length:p+length+2]
        solu1 = next_t[p]
        solu2 = next_t[p+1]
        train = []
        for j in range(len(train0)-1):
                train.append(0.5 * (train0[j] + train0[j+1]))
        solu = 0.5 * (solu1 + solu2)
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in cell average)
def gen_pair_cell_ave(u, x, t, delta_x, delta_y, length=3, num=1000):
    pairs = []
    for i in range(num):
        t_index = random.randint(0, len(t)-2)
        x_s = random.randint(0, len(x)-2*length-3)
        x_e = x_s+2*length+2
        starting = x[x_s]
        ending = x[x_e]
        input_data = []
        for i in range(x_s, x_e-1):
            value = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[0]), x[i], x[i+1])
            value = value[0] * (1/delta_x)
            input_data.append(value)
        target_data = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[t_index+1]), x[x_s+length], x[x_s+length+1])
        target_data = target_data[0] * (1/delta_x)
        pair = {'input': input_data, 'solu': target_data}
        pairs.append(pair)
    return pairs

In [5]:
# restnet
class ResNet(nn.Module):
    def __init__(self, i, h1, h2, o, twolayers):
        super(ResNet, self).__init__()
        self.linear1 = nn.Linear(i, h1)
        self.relu1 = nn.ReLU()
        self.linear21 = nn.Linear(h1, h2)
        self.relu21 = nn.ReLU()
        self.linear22 = nn.Linear(h2, o)
        self.relu22 = nn.ReLU()
        self.linear23 = nn.Linear(h1, o)
        self.relu23 = nn.ReLU()
        self.twolayers = twolayers
        
    def forward(self, x):
        if self.twolayers == True:
            out = self.linear1(x)
            out = self.relu1(out)
            out = self.linear21(out)
            out = self.relu21(out)
            out = self.linear22(out)
            out = self.relu22(out)
        else:
            out = self.linear1(x)
            out = self.relu1(out)
            out = self.linear23(out)
            out = self.relu23(out)
        return out + torch.mean(x)

    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)

In [67]:
def calc_next_time(x, t, delta_x, delta_t, current, model, length=3):
    ret = []
    padding = np.hstack(([0 for i in range(length)], current, [0 for i in range(length)])).tolist()
    padding_x = np.hstack(([0 for i in range(length)], x, [0 for i in range(length)])).tolist()
    padding_t = np.hstack(([0 for i in range(length)], t, [0 for i in range(length)])).tolist()
    for index in range(len(current)-1):
        j = index + length
        input_data = []
        print("---", j, "---")
        for i in range(j-length, j+length+1):
            value = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[0]), padding_x[i], padding_x[i+1])
            value = value[0] * (1/delta_x)
            input_data.append(value)
#         tensor_seg = torch.FloatTensor(seg)
#         out = model(tensor_seg)
#         ret.append(out.item())
#     return ret

In [68]:
x = arange(0, 2 * np.pi, 1/20)
t = arange(0, 2 * np.pi, 1/20)
X,T = meshgrid(x, t) # grid of point
Z = heat_equ_analytical_solu(X, T) # evaluation of the function on the grid
calc_next_time(x, t, 1/20, 1/20, Z[0], ResNet(7, 6, 6, 1, twolayers=False), length=3)

--- 3 ---
[0.0, 0.0, 0.0, 0.07837845807790614, 0.23320543647265152, 0.38229012305696985, 0.5219615580634472]
--- 4 ---
[0.0, 0.0, 0.07837845807790614, 0.23320543647265152, 0.38229012305696985, 0.5219615580634472, 0.6487805672193082]
--- 5 ---
[0.0, 0.07837845807790614, 0.23320543647265152, 0.38229012305696985, 0.5219615580634472, 0.6487805672193082, 0.7596244456309749]
--- 6 ---
[0.07837845807790614, 0.23320543647265152, 0.38229012305696985, 0.5219615580634472, 0.6487805672193082, 0.7596244456309749, 0.8517638491422085]
--- 7 ---
[0.23320543647265152, 0.38229012305696985, 0.5219615580634472, 0.6487805672193082, 0.7596244456309749, 0.8517638491422085, 0.9229299998454156]
--- 8 ---
[0.38229012305696985, 0.5219615580634472, 0.6487805672193082, 0.7596244456309749, 0.8517638491422085, 0.9229299998454156, 0.9713705509233703]
--- 9 ---
[0.5219615580634472, 0.6487805672193082, 0.7596244456309749, 0.8517638491422085, 0.9229299998454156, 0.9713705509233703, 0.9958927352435611]
--- 10 ---
[0.6487

[0.648780567219317, 0.5219615580634431, 0.38229012305696897, 0.23320543647264907, 0.07837845807790747, -0.07837845807790762, -0.233205436472653]
--- 102 ---
[0.5219615580634431, 0.38229012305696897, 0.23320543647264907, 0.07837845807790747, -0.07837845807790762, -0.233205436472653, -0.3822901230569676]
--- 103 ---
[0.38229012305696897, 0.23320543647264907, 0.07837845807790747, -0.07837845807790762, -0.233205436472653, -0.3822901230569676, -0.5219615580634471]
--- 104 ---
[0.23320543647264907, 0.07837845807790747, -0.07837845807790762, -0.233205436472653, -0.3822901230569676, -0.5219615580634471, -0.6487805672193048]
--- 105 ---
[0.07837845807790747, -0.07837845807790762, -0.233205436472653, -0.3822901230569676, -0.5219615580634471, -0.6487805672193048, -0.7596244456309853]
--- 106 ---
[-0.07837845807790762, -0.233205436472653, -0.3822901230569676, -0.5219615580634471, -0.6487805672193048, -0.7596244456309853, -0.8517638491422065]
--- 107 ---
[-0.233205436472653, -0.3822901230569676, -0

In [ ]:
def testing_the_model(delta_x=1/20, delta_t=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi,
                      padding_num=3, hidden=6, twolayers=False, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False,
                      model_name="model_xx delta x=1 20 delta t=1 20"):
    x = arange(xmin, xmax, delta_x)
    t = arange(ymin, ymax, delta_y)
    print("Generating meshes...")
    X,T = meshgrid(x, t) # grid of point
    print("Generating analytical solution...")
    Z = heat_equ_analytical_solu(X, T) # evaluation of the function on the grid
    print("Preparing the model and the optimizer...")
    model = ResNet(7, hidden, hidden, 1, twolayers=twolayers)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, amsgrad=amsgrad)
    criterion = nn.MSELoss()
    model.train()
    print("Updating data...")
    model.load_model(model_name)
    prediction = []
    prediction.append(torch.FloatTensor(Z[0]).tolist())
    for i in range(len(Z)-1):
        prediction.append(calc_next_time(x, t, delta_x, delta_t, Z[i], model))
        print("time", i, "is done", '", an example is:"', prediction[i+1][0])